In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from mpl_toolkits.mplot3d import axes3d
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

%matplotlib inline


# Operazioni su DataFrame

In [2]:
df_totale = pd.read_excel('dati_puliti_finali.xlsx')

In [3]:
df_totale.drop(columns=('Unnamed: 0'),inplace=True)


In [4]:
esito_12h = list()

for row in df_totale.index:
    if df_totale['sosp_foll_12h'][row] == 2 | df_totale['sosp_foll_12h'][row] == 3  :
        esito_12h.append('morto')
    else: esito_12h.append('vivo')

In [5]:
esito_24h = esito_12h.copy()

for row in df_totale.index:
    if (df_totale['sosp_foll_24h'][row] == 2 | df_totale['sosp_foll_24h'][row] == 3) | (df_totale['deg_tot_dec'][row] == 1):
        esito_24h[row] = 'morto'

In [6]:
esito_2gg = esito_24h.copy()

for row in df_totale.index:
    if (df_totale['sosp_foll_2gg'][row] == 2 | df_totale['sosp_foll_2gg'][row] == 3) | (df_totale['deg_tot_dec'][row] == 2):
        esito_2gg[row] = 'morto'

In [7]:
esito_3gg = esito_2gg.copy()

for row in df_totale.index:
    if (df_totale['sosp_foll_3gg'][row] == 2 | df_totale['sosp_foll_3gg'][row] == 3) | (df_totale['deg_tot_dec'][row] == 3):
        esito_3gg[row] = 'morto'

In [8]:
data = {'esito_12h': esito_12h, 'esito_24h': esito_24h,'esito_2gg' : esito_2gg, 'esito_3gg': esito_3gg}
df_esiti = pd.DataFrame(data=data)

In [9]:
personal_id = df_totale['record_id']

In [10]:
df_totale.drop(columns=('record_id'),inplace=True)

In [11]:
df_totale.drop(columns=('anagrafica_complete'),inplace=True)

In [12]:
df_outcome = df_totale.loc[:,'esito_ti':]

In [13]:
df_totale.drop(columns=df_outcome.columns,inplace=True)

In [14]:
df_outcome.drop(columns=('outcome_ti_complete'), inplace=True)

In [15]:
df_vis = df_totale[['vis','vis_foll_12h','vis_foll_24h','vis_foll_2gg','vis_foll_3gg']].copy()

In [16]:
df_totale.drop(columns=df_vis.columns,inplace=True)

In [17]:
media_vis = round((df_vis['vis'] + df_vis['vis_foll_12h'] + df_vis['vis_foll_24h'] +\
                   df_vis['vis_foll_2gg']+df_vis['vis_foll_3gg'])/5)
df_vis['media_vis'] = media_vis

In [18]:
df_score = df_totale[['sofa_score','ap2_total_score','saps_totale','sofa_score_foll_12h','sofa_score_foll_24h','sofa_score_foll_2gg','sofa_score_foll_3gg']].copy()

In [19]:
df_totale.drop(columns=df_score.columns, inplace=True)

In [33]:
df_score['sofa_score_foll_12h'] = df_score['sofa_score_foll_12h'].replace(0,25)
df_score['sofa_score_foll_24h'] = df_score['sofa_score_foll_24h'].replace(0,25)
df_score['sofa_score_foll_2gg'] = df_score['sofa_score_foll_2gg'].replace(0,25)
df_score['sofa_score_foll_3gg'] = df_score['sofa_score_foll_3gg'].replace(0,25)

In [34]:
delta_12h = df_score['sofa_score_foll_12h'] - df_score['sofa_score']
delta_24h = df_score['sofa_score_foll_24h'] - df_score['sofa_score_foll_12h']
delta_2gg = df_score['sofa_score_foll_2gg'] - df_score['sofa_score_foll_24h']
delta_3gg = df_score['sofa_score_foll_3gg'] - df_score['sofa_score_foll_2gg']

In [35]:
df_score['delta_12h'] = delta_12h
df_score['delta_24h'] = delta_24h
df_score['delta_2gg'] = delta_2gg
df_score['delta_3gg'] = delta_3gg

In [36]:
mean_sofa_score = round((df_score['sofa_score'] + df_score['sofa_score_foll_12h'] + df_score['sofa_score_foll_24h'] +\
                   df_score['sofa_score_foll_2gg'] + df_score['sofa_score_foll_3gg']) / 5)

In [37]:
df_score['mean_sofa'] = mean_sofa_score

In [38]:
df_score

,sofa_score,ap2_total_score,saps_totale,sofa_score_foll_12h,sofa_score_foll_24h,sofa_score_foll_2gg,sofa_score_foll_3gg,delta_12h,delta_24h,delta_2gg,delta_3gg,mean_sofa
0,11,17,41,10,8,7,10,-1,-2,-1,3,9.0
1,6,20,34,5,11,10,10,-1,6,-1,0,8.0
2,5,13,31,5,6,4,4,0,1,-2,0,5.0
3,7,17,45,6,10,9,11,-1,4,-1,2,9.0
4,11,23,56,10,9,9,9,-1,-1,0,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
270,4,21,55,4,3,4,4,0,-1,1,0,4.0
271,18,38,96,25,25,25,25,7,0,0,0,24.0
272,12,39,79,7,10,5,2,-5,3,-5,-3,7.0
273,14,22,56,13,12,11,11,-1,-1,-1,0,12.0


In [40]:
df_sepsi = df_totale[['sep','sep_foll_12h','sep_foll_24h','sep_foll_2gg','sep_foll_3gg']]

In [39]:
df_totale.drop(columns=['diff_sofa_score_foll_12h','diff_sofa_score_foll_24h','diff_sofa_score_foll_2gg',\
                        'diff_sofa_score_foll_3gg'],inplace=True)

In [41]:
df_totale.drop(columns=df_sepsi.columns,inplace=True)

In [42]:
df_horowitz = df_totale[['horowitz','horowitz_foll_12h','horowitz_foll_24h','horowitz_foll_2gg','horowitz_foll_3gg']]

In [43]:
df_totale.drop(columns=df_horowitz.columns,inplace=True)

# PCA SOLO DATI CLINICI


## Calcolo della PCA tempo 0

In [ ]:
df_clinico = df_totale.copy()

In [ ]:
df_clinico_tempo0 = df_clinico.loc[:,:'morta_saps']

In [ ]:
df_clinico_tempo0_noscore = df_clinico_tempo0.loc[:,:'wbc']

In [ ]:
df_clinico_tempo0_noscore

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(df_clinico_tempo0_noscore)
df_minmax_tempo0 = minmax_scaler.transform(df_clinico_tempo0_noscore)

In [ ]:
pca_2d = PCA(n_components=2).fit(df_minmax_tempo0)
df_pca_2d_t0 = pd.DataFrame(pca_2d.transform(df_minmax_tempo0))

In [ ]:
pca_3d = PCA(n_components=3).fit(df_minmax_tempo0)
df_pca_3d_t0 = pd.DataFrame(pca_3d.transform(df_minmax_tempo0))

In [ ]:
plt.style.use("ggplot") 
plt.plot(pca_2d.explained_variance_ratio_, marker='o', color='blue')
plt.title('PCA con 2 componenti al tempo 0')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_2d.explained_variance_ratio_)
print(np.sum(pca_2d.explained_variance_ratio_))

plt.style.use("ggplot") 
plt.plot(pca_3d.explained_variance_ratio_, marker='o', color='green')
plt.title('PCA con 3 componenti al tempo 0')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_3d.explained_variance_ratio_)
print(np.sum(pca_3d.explained_variance_ratio_))

### Distribuzione delle feature rispetto alle componenti**

A seguire vediamo come si distribuiscono le feature rispetto alle componenti, le consideriamo ordinate secondo il contributo che danno alla componente 

In [ ]:
variance_ratio_2d = pca_2d.explained_variance_ratio_
cumulative_variance_ratio_2d = np.cumsum(variance_ratio_2d)
# Get the loadings (coefficients) of each principal component
loadings_2d = pca_2d.components_

In [ ]:
variance_ratio_3d = pca_3d.explained_variance_ratio_
cumulative_variance_ratio_3d = np.cumsum(variance_ratio_3d)
# Get the loadings (coefficients) of each principal component
loadings_3d = pca_3d.components_

In [ ]:
for j in range(len(loadings_2d)):
    sorted_loadings_2d = np.argsort(np.abs(loadings_2d[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_2d:
        print("  ", df_clinico_tempo0_noscore.columns[feature_idx], ": ", loadings_2d[j][feature_idx])
    print("\n")

In [ ]:
for j in range(len(loadings_3d)):
    sorted_loadings_3d = np.argsort(np.abs(loadings_3d[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_3d:
        print("  ", df_clinico_tempo0_noscore.columns[feature_idx], ": ", loadings_3d[j][feature_idx])
    print("\n")

### Plot PCA

#### Score

In [ ]:
df_score.shape

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,12):
    
    ax = fig.add_subplot(3, 4, i+1)
    col = df_score.columns[i]
    plot2d = ax.scatter(df_pca_2d_t0.iloc[:, 0], df_pca_2d_t0.iloc[:, 1], c=df_score.loc[:,col], cmap=plt.cm.get_cmap('OrRd',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_score[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig1 = list()
for i in range(0,12):
    ax = fig.add_subplot(3, 4, i+1, projection='3d')
    col = df_score.columns[i]

    plot3d = ax.scatter(df_pca_3d_t0.iloc[:, 0], df_pca_3d_t0.iloc[:, 1],df_pca_3d_t0.iloc[:, 2],  c=df_score[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_score[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig1.append(px.scatter_3d(
        df_pca_3d_t0, x=0, y=1, z=2, color=df_score[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig1[0]

#### Output

In [ ]:
df_esiti.replace(('vivo','morto'),(1,2),inplace=True)

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,4):
    
    ax = fig.add_subplot(2, 2, i+1)
    col = df_esiti.columns[i]
    plot2d = ax.scatter(df_pca_2d_t0.iloc[:, 0], df_pca_2d_t0.iloc[:, 1], c=df_esiti.loc[:,col], cmap=plt.cm.get_cmap('OrRd',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_esiti[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig2 = list()
for i in range(0,4):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    col = df_esiti.columns[i]

    plot3d = ax.scatter(df_pca_3d_t0.iloc[:, 0], df_pca_3d_t0.iloc[:, 1],df_pca_3d_t0.iloc[:, 2],  c=df_esiti[col],\
               cmap=plt.cm.get_cmap('magma',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_esiti[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig2.append(px.scatter_3d(
        df_pca_3d_t0, x=0, y=1, z=2, color=df_esiti[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig2[1]

#### Vis

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,6):
    
    ax = fig.add_subplot(3, 2, i+1)
    col = df_vis.columns[i]
    plot2d = ax.scatter(df_pca_2d_t0.iloc[:, 0], df_pca_2d_t0.iloc[:, 1], c=df_vis.loc[:,col], cmap=plt.cm.get_cmap('cubehelix',len(np.unique(df_vis[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_vis[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig3 = list()
for i in range(0,6):
    ax = fig.add_subplot(3, 3, i+1, projection='3d')
    col = df_vis.columns[i]

    plot3d = ax.scatter(df_pca_3d_t0.iloc[:, 0], df_pca_3d_t0.iloc[:, 1],df_pca_3d_t0.iloc[:, 2],  c=df_vis[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_vis[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_vis[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig3.append(px.scatter_3d(
        df_pca_3d_t0, x=0, y=1, z=2, color=df_vis[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig3[1]

## PCA 1gg

In [ ]:
df_clinico_12h = df_clinico.iloc[:,df_clinico.columns.get_loc('occhi_gcs_foll_12h'):df_clinico.columns.get_loc('diff_sofa_score_foll_12h')]

In [ ]:
df_clinico_24h= df_clinico.iloc[:,df_clinico.columns.get_loc('occhi_gcs_foll_24h'):df_clinico.columns.get_loc('diff_sofa_score_foll_24h')]

In [ ]:
df_clinico_1gg = pd.merge(df_clinico_tempo0,df_clinico_12h,right_index=True, left_index=True)

In [ ]:
df_clinico_1gg = pd.merge(df_clinico_1gg,df_clinico_24h,right_index=True, left_index=True)

In [ ]:
df_clinico_1gg

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(df_clinico_1gg)
df_minmax_1gg = minmax_scaler.transform(df_clinico_1gg)

In [ ]:
pca_2d_1gg = PCA(n_components=2).fit(df_minmax_1gg)
df_pca_2d_1gg = pd.DataFrame(pca_2d_1gg.transform(df_minmax_1gg))

In [ ]:
pca_3d_1gg = PCA(n_components=3).fit(df_minmax_1gg)
df_pca_3d_1gg = pd.DataFrame(pca_3d_1gg.transform(df_minmax_1gg))

In [ ]:
plt.style.use("ggplot") 
plt.plot(pca_2d_1gg.explained_variance_ratio_, marker='o', color='orange')
plt.title('PCA con 2 componenti - giorno 1')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_2d_1gg.explained_variance_ratio_)
print(np.sum(pca_2d_1gg.explained_variance_ratio_))

plt.style.use("ggplot") 
plt.plot(pca_3d_1gg.explained_variance_ratio_, marker='o', color='purple')
plt.title('PCA con 3 componenti - 1 giorno')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_3d_1gg.explained_variance_ratio_)
print(np.sum(pca_3d_1gg.explained_variance_ratio_))

### Distribuzione delle feature rispetto alle componenti

A seguire vediamo come si distribuiscono le feature rispetto alle componenti, le consideriamo ordinate secondo il contributo che danno alla componente 

In [ ]:
variance_ratio_2d_1gg = pca_2d_1gg.explained_variance_ratio_
cumulative_variance_ratio_2d_1gg = np.cumsum(variance_ratio_2d_1gg)
# Get the loadings (coefficients) of each principal component
loadings_2d_1gg = pca_2d_1gg.components_

In [ ]:
variance_ratio_3d_1gg = pca_3d_1gg.explained_variance_ratio_
cumulative_variance_ratio_3d_1gg = np.cumsum(variance_ratio_3d_1gg)
# Get the loadings (coefficients) of each principal component
loadings_3d_1gg = pca_3d_1gg.components_

In [ ]:
for j in range(len(loadings_2d_1gg)):
    sorted_loadings_2d_1gg = np.argsort(np.abs(loadings_2d_1gg[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_2d_1gg:
        print("  ", df_clinico_1gg.columns[feature_idx], ": ", loadings_2d_1gg[j][feature_idx])
    print("\n")

In [ ]:
for j in range(len(loadings_3d_1gg)):
    sorted_loadings_3d_1gg = np.argsort(np.abs(loadings_3d_1gg[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_3d_1gg:
        print("  ", df_clinico_1gg.columns[feature_idx], ": ", loadings_3d_1gg[j][feature_idx])
    print("\n")

### Plot PCA

#### Score

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,12):
    
    ax = fig.add_subplot(3, 4, i+1)
    col = df_score.columns[i]
    plot2d = ax.scatter(df_pca_2d_1gg.iloc[:, 0], df_pca_2d_1gg.iloc[:, 1], c=df_score.loc[:,col], cmap=plt.cm.get_cmap('OrRd',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_score[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig4 = list()
for i in range(0,12):
    ax = fig.add_subplot(3, 4, i+1, projection='3d')
    col = df_score.columns[i]

    plot3d = ax.scatter(df_pca_3d_1gg.iloc[:, 0], df_pca_3d_1gg.iloc[:, 1],df_pca_3d_1gg.iloc[:, 2],  c=df_score[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_score[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig4.append(px.scatter_3d(
        df_pca_3d_1gg, x=0, y=1, z=2, color=df_score[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig4[5]

#### Output

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,4):
    
    ax = fig.add_subplot(2, 2, i+1)
    col = df_esiti.columns[i]
    plot2d = ax.scatter(df_pca_2d_1gg.iloc[:, 0], df_pca_2d_1gg.iloc[:, 1], c=df_esiti.loc[:,col], cmap=plt.cm.get_cmap('OrRd',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_esiti[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig5 = list()
for i in range(0,4):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    col = df_esiti.columns[i]

    plot3d = ax.scatter(df_pca_3d_1gg.iloc[:, 0], df_pca_3d_1gg.iloc[:, 1],df_pca_3d_1gg.iloc[:, 2],  c=df_esiti[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_esiti[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig5.append(px.scatter_3d(
        df_pca_3d_1gg, x=0, y=1, z=2, color=df_esiti[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig5[1]

#### Vis

In [ ]:

fig = plt.figure(figsize=(40,30))

fig6 = list()
for i in range(0,6):
    ax = fig.add_subplot(3, 3, i+1, projection='3d')
    col = df_vis.columns[i]

    plot3d = ax.scatter(df_pca_3d_1gg.iloc[:, 0], df_pca_3d_1gg.iloc[:, 1],df_pca_3d_1gg.iloc[:, 2],  c=df_vis[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_vis[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_vis[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig6.append(px.scatter_3d(
        df_pca_3d_1gg, x=0, y=1, z=2, color=df_vis[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig6[5]

## PCA 2gg

In [ ]:
df_clinico_2 = df_clinico.iloc[:,df_clinico.columns.get_loc('occhi_gcs_foll_2gg'):df_clinico.columns.get_loc('diff_sofa_score_foll_2gg')]

In [ ]:
df_clinico_2gg = pd.merge(df_clinico_1gg,df_clinico_2,right_index=True, left_index=True)

In [ ]:
df_clinico_2gg

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(df_clinico_2gg)
df_minmax_2gg = minmax_scaler.transform(df_clinico_2gg)

In [ ]:
pca_2d_2gg = PCA(n_components=2).fit(df_minmax_2gg)
df_pca_2d_2gg = pd.DataFrame(pca_2d_2gg.transform(df_minmax_2gg))

In [ ]:
pca_3d_2gg = PCA(n_components=3).fit(df_minmax_2gg)
df_pca_3d_2gg = pd.DataFrame(pca_3d_2gg.transform(df_minmax_2gg))

In [ ]:
plt.style.use("ggplot") 
plt.plot(pca_2d_2gg.explained_variance_ratio_, marker='o', color='orange')
plt.title('PCA con 2 componenti - giorno 2')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_2d_2gg.explained_variance_ratio_)
print(np.sum(pca_2d_2gg.explained_variance_ratio_))

plt.style.use("ggplot") 
plt.plot(pca_3d_2gg.explained_variance_ratio_, marker='o', color='purple')
plt.title('PCA con 3 componenti - 2 giorno')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_3d_2gg.explained_variance_ratio_)
print(np.sum(pca_3d_2gg.explained_variance_ratio_))

### Distribuzione delle feature rispetto alle componenti

A seguire vediamo come si distribuiscono le feature rispetto alle componenti, le consideriamo ordinate secondo il contributo che danno alla componente 

In [ ]:
variance_ratio_2d_2gg = pca_2d_2gg.explained_variance_ratio_
cumulative_variance_ratio_2d_2gg = np.cumsum(variance_ratio_2d_2gg)
# Get the loadings (coefficients) of each principal component
loadings_2d_2gg = pca_2d_2gg.components_

In [ ]:
variance_ratio_3d_2gg = pca_3d_2gg.explained_variance_ratio_
cumulative_variance_ratio_3d_2gg = np.cumsum(variance_ratio_3d_2gg)
# Get the loadings (coefficients) of each principal component
loadings_3d_2gg = pca_3d_2gg.components_

In [ ]:
for j in range(len(loadings_2d_2gg)):
    sorted_loadings_2d_2gg = np.argsort(np.abs(loadings_2d_2gg[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_2d_2gg:
        print("  ", df_clinico_2gg.columns[feature_idx], ": ", loadings_2d_2gg[j][feature_idx])
    print("\n")

In [ ]:
for j in range(len(loadings_3d_2gg)):
    sorted_loadings_3d_2gg = np.argsort(np.abs(loadings_3d_2gg[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_3d_2gg:
        print("  ", df_clinico_2gg.columns[feature_idx], ": ", loadings_3d_2gg[j][feature_idx])
    print("\n")

### Plot PCA

#### Score

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,12):
    
    ax = fig.add_subplot(3, 4, i+1)
    col = df_score.columns[i]
    plot2d = ax.scatter(df_pca_2d_2gg.iloc[:, 0], df_pca_2d_2gg.iloc[:, 1], c=df_score.loc[:,col], cmap=plt.cm.get_cmap('OrRd',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_score[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig7 = list()
for i in range(0,12):
    ax = fig.add_subplot(3, 4, i+1, projection='3d')
    col = df_score.columns[i]

    plot3d = ax.scatter(df_pca_3d_2gg.iloc[:, 0], df_pca_3d_2gg.iloc[:, 1],df_pca_3d_2gg.iloc[:, 2],  c=df_score[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_score[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig7.append(px.scatter_3d(
        df_pca_3d_2gg, x=0, y=1, z=2, color=df_score[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig7[11]

#### Output

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,4):
    
    ax = fig.add_subplot(2, 2, i+1)
    col = df_esiti.columns[i]
    plot2d = ax.scatter(df_pca_2d_2gg.iloc[:, 0], df_pca_2d_2gg.iloc[:, 1], c=df_esiti.loc[:,col], cmap=plt.cm.get_cmap('Spectral',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_esiti[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig8 = list()
for i in range(0,4):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    col = df_esiti.columns[i]

    plot3d = ax.scatter(df_pca_3d_2gg.iloc[:, 0], df_pca_3d_2gg.iloc[:, 1],df_pca_3d_2gg.iloc[:, 2],  c=df_esiti[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_esiti[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig8.append(px.scatter_3d(
        df_pca_3d_2gg, x=0, y=1, z=2, color=df_esiti[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig8[2]

#### Vis

In [ ]:

fig = plt.figure(figsize=(40,30))

fig9 = list()
for i in range(0,6):
    ax = fig.add_subplot(3, 3, i+1, projection='3d')
    col = df_vis.columns[i]

    plot3d = ax.scatter(df_pca_3d_2gg.iloc[:, 0], df_pca_3d_2gg.iloc[:, 1],df_pca_3d_2gg.iloc[:, 2],  c=df_vis[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_vis[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_vis[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig9.append(px.scatter_3d(
        df_pca_3d_2gg, x=0, y=1, z=2, color=df_vis[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig9[5]

## PCA 3 giorni

In [ ]:
df_clinico

In [ ]:
df_clinico_3 = df_clinico.iloc[:,df_clinico.columns.get_loc('occhi_gcs_foll_3gg'):df_clinico.columns.get_loc('diff_sofa_score_foll_3gg')]

In [ ]:
df_clinico_3gg = pd.merge(df_clinico_2gg,df_clinico_3,right_index=True, left_index=True)

In [ ]:
df_clinico_3gg

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(df_clinico_3gg)
df_minmax_3gg = minmax_scaler.transform(df_clinico_3gg)

In [ ]:
pca_2d_3gg = PCA(n_components=2).fit(df_minmax_3gg)
df_pca_2d_3gg = pd.DataFrame(pca_2d_3gg.transform(df_minmax_3gg))

In [ ]:
pca_3d_3gg = PCA(n_components=3).fit(df_minmax_3gg)
df_pca_3d_3gg = pd.DataFrame(pca_3d_3gg.transform(df_minmax_3gg))

In [ ]:
plt.style.use("ggplot") 
plt.plot(pca_2d_3gg.explained_variance_ratio_, marker='o', color='violet')
plt.title('PCA con 2 componenti - giorno 3')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_2d_3gg.explained_variance_ratio_)
print(np.sum(pca_2d_3gg.explained_variance_ratio_))

plt.style.use("ggplot") 
plt.plot(pca_3d_3gg.explained_variance_ratio_, marker='o', color='pink')
plt.title('PCA con 3 componenti - 3 giorno')
plt.xlabel('ordered PCs')
plt.ylabel('PVE (%)')
plt.show()
print(pca_3d_3gg.explained_variance_ratio_)
print(np.sum(pca_3d_3gg.explained_variance_ratio_))

### Distribuzione delle feature rispetto alle componenti

A seguire vediamo come si distribuiscono le feature rispetto alle componenti, le consideriamo ordinate secondo il contributo che danno alla componente 

In [ ]:
variance_ratio_2d_3gg = pca_2d_3gg.explained_variance_ratio_
cumulative_variance_ratio_2d_3gg = np.cumsum(variance_ratio_2d_3gg)
# Get the loadings (coefficients) of each principal component
loadings_2d_3gg = pca_2d_3gg.components_

In [ ]:
variance_ratio_3d_3gg = pca_3d_3gg.explained_variance_ratio_
cumulative_variance_ratio_3d_3gg = np.cumsum(variance_ratio_3d_3gg)
# Get the loadings (coefficients) of each principal component
loadings_3d_3gg = pca_3d_3gg.components_

In [ ]:
for j in range(len(loadings_2d_3gg)):
    sorted_loadings_2d_3gg = np.argsort(np.abs(loadings_2d_3gg[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_2d_3gg:
        print("  ", df_clinico_3gg.columns[feature_idx], ": ", loadings_2d_3gg[j][feature_idx])
    print("\n")

In [ ]:
for j in range(len(loadings_3d_3gg)):
    sorted_loadings_3d_3gg = np.argsort(np.abs(loadings_3d_3gg[j]))[::-1]
    print("Features with highest loadings in Principal Component", j+1,"\n")
    for feature_idx in sorted_loadings_3d_3gg:
        print("  ", df_clinico_3gg.columns[feature_idx], ": ", loadings_3d_3gg[j][feature_idx])
    print("\n")

### Plot PCA

#### Score

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,12):
    
    ax = fig.add_subplot(3, 4, i+1)
    col = df_score.columns[i]
    plot2d = ax.scatter(df_pca_2d_3gg.iloc[:, 0], df_pca_2d_3gg.iloc[:, 1], c=df_score.loc[:,col], cmap=plt.cm.get_cmap('OrRd',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_score[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig10 = list()
for i in range(0,12):
    ax = fig.add_subplot(3, 4, i+1, projection='3d')
    col = df_score.columns[i]

    plot3d = ax.scatter(df_pca_3d_3gg.iloc[:, 0], df_pca_3d_3gg.iloc[:, 1],df_pca_3d_3gg.iloc[:, 2],  c=df_score[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(df_score[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_score[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig10.append(px.scatter_3d(
        df_pca_3d_3gg, x=0, y=1, z=2, color=df_score[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig10[11]

#### Output

In [ ]:
fig = plt.figure(figsize=(30,20))

for i in range(0,4):
    
    ax = fig.add_subplot(2, 2, i+1)
    col = df_esiti.columns[i]
    plot2d = ax.scatter(df_pca_2d_3gg.iloc[:, 0], df_pca_2d_3gg.iloc[:, 1], c=df_esiti.loc[:,col], cmap=plt.cm.get_cmap('tab20b',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot2d , ticks=range(len(np.unique(df_esiti[col]))+1), label=col )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:

fig = plt.figure(figsize=(40,30))

fig11 = list()
for i in range(0,4):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    col = df_esiti.columns[i]

    plot3d = ax.scatter(df_pca_3d_3gg.iloc[:, 0], df_pca_3d_3gg.iloc[:, 1],df_pca_3d_3gg.iloc[:, 2],  c=df_esiti[col],\
               cmap=plt.cm.get_cmap('tab20b',len(np.unique(df_esiti[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_esiti[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig11.append(px.scatter_3d(
        df_pca_3d_3gg, x=0, y=1, z=2, color=df_esiti[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig11[3]

#### Vis

In [ ]:

fig = plt.figure(figsize=(40,30))

fig12 = list()
for i in range(0,6):
    ax = fig.add_subplot(3, 3, i+1, projection='3d')
    col = df_vis.columns[i]

    plot3d = ax.scatter(df_pca_3d_3gg.iloc[:, 0], df_pca_3d_3gg.iloc[:, 1],df_pca_3d_3gg.iloc[:, 2],  c=df_vis[col],\
               cmap=plt.cm.get_cmap('tab20b',len(np.unique(df_vis[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(df_vis[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig12.append(px.scatter_3d(
        df_pca_3d_3gg, x=0, y=1, z=2, color=df_vis[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig12[5]

## Extra

### Anticoagulanti usati in trattamento

In [ ]:
anticrrt = df_totale[['antic_crrt','antic_crrt_foll_12h','antic_crrt_foll_24h','antic_crrt_foll_2gg','antic_crrt_foll_3gg']]

In [ ]:

fig = plt.figure(figsize=(40,30))

fig13 = list()
for i in range(0,5):
    ax = fig.add_subplot(3, 3, i+1, projection='3d')
    col = anticrrt.columns[i]

    plot3d = ax.scatter(df_pca_3d_3gg.iloc[:, 0], df_pca_3d_3gg.iloc[:, 1],df_pca_3d_3gg.iloc[:, 2],  c=anticrrt[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(anticrrt[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(anticrrt[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig13.append(px.scatter_3d(
        df_pca_3d_3gg, x=0, y=1, z=2, color=anticrrt[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig13[0]

### Presenza o meno di sepsi

In [ ]:
sepsi = df_totale[['sep','sep_foll_12h','sep_foll_24h','sep_foll_2gg','sep_foll_3gg']]

In [ ]:
fig = plt.figure(figsize=(40,30))

fig14 = list()
for i in range(0,5):
    ax = fig.add_subplot(3, 3, i+1, projection='3d')
    col = sepsi.columns[i]

    plot3d = ax.scatter(df_pca_3d_t0.iloc[:, 0], df_pca_3d_t0.iloc[:, 1],df_pca_3d_t0.iloc[:, 2],  c=sepsi[col],\
               cmap=plt.cm.get_cmap('RdPu',len(np.unique(sepsi[col]))))
    plt.xlabel('PC-1')
    plt.ylabel('PC-2')
    fig.colorbar(plot3d ,ticks=range(len(np.unique(sepsi[col]))+1), label=col)
    plt.title(str(i) + " - " + str(col))
    
    fig14.append(px.scatter_3d(
        df_pca_3d_t0, x=0, y=1, z=2, color=sepsi[col],
        color_continuous_scale='magenta',title=str(col),
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )
    )
    
plt.rcParams['axes.grid'] = False

plt.show()

In [ ]:
fig14[1]

# K-mean

## Tempo 0

In [ ]:
K=2
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_tempo0)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_tempo0, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_tempo0, y)

print(f'overall clustering silhouette {silhouette_avg}')

fig, ax = plt.subplots(1,2,figsize=(15,5))

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    ax[0].barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    ax[0].text(-0.03, (y_lower+y_upper)/2, str(i))
    ax[0].text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

ax[0].axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)

ax[1].scatter(df_pca_2d_t0.iloc[:, 0], df_pca_2d_t0.iloc[:, 1], c=colors)


In [ ]:
K=2
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_tempo0)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_tempo0, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_tempo0, y)

print(f'overall clustering silhouette {silhouette_avg}')

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    plt.barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    plt.text(-0.03, (y_lower+y_upper)/2, str(i))
    plt.text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

plt.axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)
plt.show()

px.scatter_3d(
        df_pca_3d_t0, x=0, y=1, z=2, color=y,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )

## 1 Giorno

In [ ]:
K=2
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_1gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_1gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_1gg, y)

print(f'overall clustering silhouette {silhouette_avg}')

fig, ax = plt.subplots(1,2,figsize=(15,5))

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    ax[0].barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    ax[0].text(-0.03, (y_lower+y_upper)/2, str(i))
    ax[0].text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

ax[0].axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)

ax[1].scatter(df_pca_2d_1gg.iloc[:, 0], df_pca_2d_1gg.iloc[:, 1], c=colors)


In [ ]:
K=3
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_1gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_1gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_1gg, y)

print(f'overall clustering silhouette {silhouette_avg}')


y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    plt.barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    plt.text(-0.03, (y_lower+y_upper)/2, str(i))
    plt.text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

plt.axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)
plt.show()

px.scatter_3d(
        df_pca_3d_1gg, x=0, y=1, z=2, color=y,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )

## 2 Giorni

In [ ]:
K=2
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_2gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_2gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_2gg, y)

print(f'overall clustering silhouette {silhouette_avg}')

fig, ax = plt.subplots(1,2,figsize=(15,5))

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    ax[0].barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    ax[0].text(-0.03, (y_lower+y_upper)/2, str(i))
    ax[0].text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

ax[0].axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)

ax[1].scatter(df_pca_2d_2gg.iloc[:, 0], df_pca_2d_2gg.iloc[:, 1], c=colors)


In [ ]:
K=3
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_2gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_2gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_2gg, y)

print(f'overall clustering silhouette {silhouette_avg}')

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    plt.barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    plt.text(-0.03, (y_lower+y_upper)/2, str(i))
    plt.text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

plt.axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)
plt.show()

px.scatter_3d(
        df_pca_3d_2gg, x=0, y=1, z=2, color=y,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )

## 3 Giorni

In [ ]:
K=2
kmeans_model = KMeans(n_clusters=K, init='random', n_init=10)
kmeans_model.fit(df_minmax_3gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_3gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_3gg, y)

print(f'overall clustering silhouette {silhouette_avg}')

fig, ax = plt.subplots(1,2,figsize=(15,5))

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    ax[0].barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    ax[0].text(-0.03, (y_lower+y_upper)/2, str(i))
    ax[0].text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

ax[0].axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)

ax[1].scatter(df_pca_2d_3gg.iloc[:, 0], df_pca_2d_3gg.iloc[:, 1], c=colors)


In [ ]:
K=3
kmeans_model = KMeans(n_clusters=K, init='random', n_init=30)
kmeans_model.fit(df_minmax_3gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_3gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_3gg, y)

print(f'overall clustering silhouette {silhouette_avg}')

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    plt.barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    plt.text(-0.03, (y_lower+y_upper)/2, str(i))
    plt.text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

plt.axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)
plt.show()

px.scatter_3d(
        df_pca_3d_3gg, x=0, y=1, z=2, color=y,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )

In [ ]:
K=2
kmeans_model = KMeans(n_clusters=K, init='random', n_init=30)
kmeans_model.fit(df_minmax_3gg)
y = kmeans_model.labels_


silhouette_avg = silhouette_score(df_minmax_3gg, y) #silhouette media
sample_silhouette_values = silhouette_samples(df_minmax_3gg, y)

print(f'overall clustering silhouette {silhouette_avg}')

y_lower = 0 
y_upper = 0
for i,cluster in enumerate(np.unique(y)):
    cluster_silhouette_vals = sample_silhouette_values[y == cluster]
    cluster_silhouette_vals.sort()
    cluster_silhouette_mean=cluster_silhouette_vals.mean()
    y_upper += len(cluster_silhouette_vals)
    color = cm.nipy_spectral(float(i) / K)
    plt.barh(range(y_lower,y_upper), cluster_silhouette_vals, height=1, color=color)
    plt.text(-0.03, (y_lower+y_upper)/2, str(i))
    plt.text(silhouette_avg+0.03, y_lower, cluster_silhouette_mean)  
    y_lower += len(cluster_silhouette_vals)

plt.axvline(silhouette_avg, linestyle ='--', linewidth =2, color = 'red')
colors = cm.nipy_spectral(y.astype(float) / K)
plt.show()

px.scatter_3d(
        df_pca_3d_3gg, x=0, y=1, z=2, color=y,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
        height=700
    )

**DA FARE**

1. Percentuale mortalità rispetto al cluster
2. Percentuale di mortalità a pari del sofa score e sepsi
3. Ascisse feature ordinate valore percentuale sulle prime 30 regolarsi sul calo percentuale